# Getting Started with using Anthropic models using Amazon Bedrock

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio on ml.t3.medium instance*

---

In this demo notebook, we demonstrate how to use the boto3 Python SDK to work with Amazon Bedrock Foundation Models. If you are running this in AWS provided accounts, excessive API calls to Bedrock APIs may results in throttling and your account may get blocked

---

You can now access Claude, the latest version of Anthropic’s large language model (LLM), on Amazon Bedrock. Claude can take up to 200,000 tokens in each prompt, meaning it can work over hundreds of pages of text, or even an entire book. Claude 2 can also write longer documents—on the order of a few thousand tokens—compared to its prior version, giving you even greater ways to develop generative AI applications using Amazon Bedrock.

Anthropic, an AI safety and research lab that builds reliable, interpretable, and steerable AI systems, is the maker of the state-of-the art LLM, Claude. The new version of the LLM, Claude 3, can process large amounts of text. With its 200,000 token context window, which is equivalent to about 200 pages of information, Claude lets you process large amounts of data in a single prompt. As a result, you can use documents, emails, FAQs, chat transcripts or even entire codebases as inputs for Claude to edit, rewrite, summarize, answer questions, generate code, and more.

## Imports and set up

In [ ]:
import sys
import os
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils.environment_validation import validate_environment, validate_model_access
validate_environment()

In [ ]:
required_models = [
    "amazon.titan-embed-text-v1",
    "anthropic.claude-3-sonnet-20240229-v1:0",
    "anthropic.claude-3-haiku-20240307-v1:0",
]
validate_model_access(required_models)

## Set Up Bedrock Client

To help with this, we've provided a get_bedrock_client() utility method that supports passing in different options. You can find the implementation in ../utils/bedrock.py

The get_bedrock_client() method accepts runtime (default=True) parameter to return either bedrock or bedrock-runtime client.

In [ ]:

import json
import re

import boto3

from utils import bedrock, print_ww
from utils.prompt_utils import prompts_to_messages, convert_pdf_to_image, convert_pil_image_to_b64
from rich import print as rprint
from rich.markdown import Markdown


bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)


## The Great Gatsby
We will explore the large context capabilities of Claude by using the entire text of the book "The Great Gatsby" by F. Scott Fitzgerald as input. The Great Gatsby is a novel by American writer F. Scott Fitzgerald. 

In [ ]:

from pypdf import PdfReader

reader = PdfReader("./data/the-great-gatsby.pdf")
number_of_pages = len(reader.pages)
text = ''.join([page.extract_text() for page in reader.pages])
print(text[:1000])


In [ ]:

total_no_of_words_input = len(text.split())
# total_no_of_tokens = total_no_of_words/0.75
print(f"Total Number of Pages in Great Gatsby Book: {number_of_pages}")
print(f"Total Number of words in Great Gatsby Book: {total_no_of_words_input}")
# print(f"Total Number of tokens in Great Gatsby Book: {total_no_of_tokens}")


## Let's get create a prompt and invoke Claude 3 Haiku
In this example we will provide the first two chapters of the book to Claude and ask it to generate a list and description of each of the main characters in the book. As this is a widely known novel, Claude should be able to provide a good summary of the characters without any external context. However, this is not what we want in this case as we want to have Claude use the context provided to generate the output. We will therefore include a specific instruction to Claude to use the context provided in the prompt.

Try experimenting yourself to see what happens if you remove the instruction to only use the context provided in the prompt.

In [ ]:

# If you'd like to try your own prompt, edit this parameter!

two_chapters = text[:text.index("Chapter 3")]

prompt = f"""Here is The Great Gatsby Book: 
<book>
{two_chapters}
</book>

I would like to know the list all the characters from this book. Describe the main characters and their roles in this book.
Only use the text provided above to generate the list of characters, do not use any external sources or your own knowledge.

"""

body = json.dumps({
    "max_tokens": 1024,
    "messages": prompts_to_messages(prompt),
    "anthropic_version": "bedrock-2023-05-31"
})

modelId = "anthropic.claude-3-haiku-20240307-v1:0"  # change this to use a different version from the model provider
# modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
# modelId = "anthropic.claude-v2:1" 
accept = "application/json"
contentType = "application/json"


In [ ]:

import time

# get the start time
st = time.time()

response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)

et = time.time()

elapsed_time = et - st

print('Execution time:', elapsed_time, 'seconds')


response_body = json.loads(response.get("body").read())
response_completion = response_body.get("content")[0]["text"]
rprint(response_completion)


### On-Demand Pricing calcualtion

Please refer https://aws.amazon.com/bedrock/pricing/ for latest on-demand provisioned throughput pricing details

In [ ]:

total_no_words_output = len(response_completion.split())
 
total_no_of_tokens_input = response['ResponseMetadata']['HTTPHeaders']['x-amzn-bedrock-input-token-count']
total_no_tokens_output = response['ResponseMetadata']['HTTPHeaders']['x-amzn-bedrock-output-token-count']
total_response_time = response['ResponseMetadata']['HTTPHeaders']['x-amzn-bedrock-invocation-latency']



print(f"Total Number of words in Great Gatsby Book: {total_no_of_words_input}")
print(f"Total Number of tokens in Great Gatsby Book: {total_no_of_tokens_input}")
print(f"Total Number of words in Output: {total_no_words_output}")
print(f"Total Number of tokens in Output: {total_no_tokens_output}")

print('Execution time:', int(total_response_time)/1000, 'seconds')

cost_of_input_tokens = int(total_no_of_tokens_input) * 0.00025 / 1000 #on-demand pricing per input token for Claude 3 Haiku
cost_of_output_tokens = int(total_no_tokens_output) * 0.00125 /100 #on-demand pricing per output token for Claude 3 Haiku
total_cost = cost_of_input_tokens + cost_of_output_tokens

print(f"Total cost for input tokens: {cost_of_input_tokens:.6f}")
print(f"Total cost of output tokens: {cost_of_output_tokens:.6f}")
print(f"Total cost: ${total_cost:.6f}")


### Let's load Amazon's annual report and analyze the document

In [ ]:

from pypdf import PdfReader

amazon_reader = PdfReader("./data/Amazon-2022-Annual-Report.pdf")
amazon_number_of_pages = len(amazon_reader.pages)
amazon_text = ''.join([page.extract_text() for page in amazon_reader.pages])
print(amazon_text[:1000])


## Set up prompt template with following best practices

Anthropic provides a number of best practices for creating prompts that can help you get the best results from Claude.  These include:
- Be clear and direct
- Provide examples (i.e. multi-shot learning)
- Let Claude think (Chain of Thought prompting)
- Use XML tags
- Provide a system prompt
- Prefill Claude's response

We will explore some of these best practices in this notebook but always refer to the [Anthropic documentation](https://docs.anthropic.com/en/docs/build-with-claude/prompt-engineering/overview) for the most up-to-date information.

In [ ]:

# If you'd like to try your own prompt, edit this parameter!
prompt_data = f"""Here is Amazon financial annual report: 
<report>
{amazon_text}
</report>

You are a helpful financial analyst. Please do the following:
1. Summarize the annual report and provide highlights in bullet points. Place output into <summary></summary> tags.)
2. What is the total value creation in 2022?
3. What is the total value creation in 2020?
4. How was year 2020 different from 2022 for the shareholders?


"""


### Invoke Claude model using Bedrock API

The Anthropic Claude models support the following parameters to control the length of the generated response.

Maximum length (max_tokens_to_sample) – Specify the maximum number of tokens to use in the generated response.

Stop sequences (stop_sequences) – Configure up to four sequences that the model recognizes. After a stop sequence, the model stops generating further tokens. The returned text doesn't contain the stop sequence.

In [ ]:

body = json.dumps({
    "max_tokens": 1024,
    "messages": prompts_to_messages(prompt_data),
    "anthropic_version": "bedrock-2023-05-31"
})

modelId = "anthropic.claude-3-haiku-20240307-v1:0"  # change this to use a different version from the model provider
# modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
# modelId = "anthropic.claude-v2:1" 
accept = "application/json"
contentType = "application/json"

response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("content")[0]["text"])


## Prompt guidance for different NLP tasks

### Information Extraction
A common use case for LLMs is to extract information from a document. To do this, we often want the model to provide a consistent output in a specified json format. We can accomplish this with Claude by feeding it our target schema as part of the prompt. Rather than having to manually craft the json schema, it is often better to utilize the [pydantic](https://docs.pydantic.dev/latest/) library to generate the schema for us from a data model that we design with Python classes.

```

In [ ]:

# If you'd like to try your own prompt, edit this parameter!
from pydantic.v1 import BaseModel, Field
from typing import List

# first we define a data model for the executive officer containing the information we want to extract
# In this case we want to extract the name, age, position and description of the executive officer
class ExecutiveOfficer(BaseModel):
    name: str = Field(..., description="Name of the executive officer")
    age: str = Field(..., description="Age of the executive officer")
    position: str = Field(..., description="Position held by the executive officer")
    description: str = Field(..., description="Description about the executive officer")

# since there are multiple executive officers, we define a data model for the list of executive officers
class ExecutiveOfficers(BaseModel):
    executive_officers: List[ExecutiveOfficer] = Field(..., description="A list of executive officers")
    

# next we generate a schema that we can feed to the prompt
schema = {k: v for k, v in ExecutiveOfficers.schema().items()}


# Finally we create the prompt where we explain how to interpret the schema and what information to extract
information_extraction_template = f"""Here is Amazon financial annual report: 
<report>
{amazon_text}
</report>

 Please precisely extract information about executive officers.

1. Name of the executive officer
2. Age
3. Position held
4. Description about each of the officer

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {{"properties": {{"foo": {{"title": "Foo", "description": "a list of strings", "type": "array", "items": {{"type": "string"}}}}}}, "required": ["foo"]}}
the object {{"foo": ["bar", "baz"]}} is a well-formatted instance of the schema. The object {{"properties": {{"foo": ["bar", "baz"]}}}} is not well-formatted.

Here is the output schema:
<schema>
{schema}
</schema>

Place the output in <output></output> tags.

"""


In [ ]:

body = json.dumps({
    "max_tokens": 1024,
    "messages": [{"role": "user", "content": information_extraction_template}],
    "anthropic_version": "bedrock-2023-05-31"
})


modelId = "anthropic.claude-3-haiku-20240307-v1:0"  # change this to use a different version from the model provider
# modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
# modelId = "anthropic.claude-v2:1" 
accept = "application/json"
contentType = "application/json"

response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

# since we instructed the model to output the JSON instance in the <output></output> tags, we can easily extract it with a regular expression
json_output = re.findall(r'<output>(.*?)</output>', response_body.get("content")[0]["text"], re.DOTALL)[0]

# we can also validate the output against the schema. In more complex LLM chains, we can use the output of a failed validation to have the model correct the output
try:
    ExecutiveOfficers.parse_raw(json_output)
    print(f"Output is valid JSON and conforms to the schema")
except Exception as e:
    print(f"Error parsing output: {e}")
    
rprint(Markdown(f'```json\n{json_output}```'))


### Information Extraction from Images
Information extraction is not limited only to text based data. We can also extract information from images using the same approach. Let's revisit the JPMC annual report example from the prior notebook but this time we'll use pydantic to generate a more robust schema for the extracted information.
```

In [ ]:
import fitz

# First we read the PDF file
# Then we convert the PDF to an image

pdf_path = "data/jpmc_annual_report_page_6.pdf"
doc = fitz.open(pdf_path)
img = convert_pdf_to_image(doc, page_number=0, dpi=150)
img

In [ ]:
# class to define the data points for each year. This will be used to define the data points for all of the annual metrics
class DataPoint(BaseModel):
    year: str = Field(..., description="Year of the data point")
    value: float = Field(..., description="Value of the data point")

# class to define the annual metrics including the units and data points
class AnnualMetrics(BaseModel):
    units: str = Field(..., description="Units of the data such as $B, %, etc.")
    data_points: List[DataPoint] = Field(..., description="List of data points")

# finally we define the financial metrics that include the net income, diluted EPS, and ROTCE
class FinancialMetrics(BaseModel):
    net_income: AnnualMetrics = Field(..., description="Net income data")
    diluted_eps: AnnualMetrics = Field(..., description="Diluted earnings per share (EPS) data")
    rotce: AnnualMetrics = Field(..., description="Return on average tangible common equity (ROTCE) data")
    

# next we generate a schema that we can feed to the prompt
schema = {k: v for k, v in FinancialMetrics.schema().items()}

text_prompt = f"""
Extract the data contained in the bar graph and provide it in a json format. The data should include the Net Income, Diluted EPS, and ROTCE from 2004 to 2022.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {{"properties": {{"foo": {{"title": "Foo", "description": "a list of strings", "type": "array", "items": {{"type": "string"}}}}}}, "required": ["foo"]}}
the object {{"foo": ["bar", "baz"]}} is a well-formatted instance of the schema. The object {{"properties": {{"foo": ["bar", "baz"]}}}} is not well-formatted.

Here is the target output schema:
{schema}
Do not include the schema in the output, just generate the JSON instance.

Place the output in <output></output> tags.

"""
image_prompt = convert_pil_image_to_b64(img)

In [ ]:
body = json.dumps({
    "max_tokens": 2048,
    "messages": prompts_to_messages([{"role": "user", "text_prompt": text_prompt, "image_prompt": image_prompt}]),
    "anthropic_version": "bedrock-2023-05-31"
})

modelId = "anthropic.claude-3-sonnet-20240229-v1:0"  
# modelId = "anthropic.claude-3-haiku-20240307-v1:0"
# modelId = "anthropic.claude-v2:1" 
accept = "application/json"
contentType = "application/json"


response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
json_output = re.findall(r'<output>(.*?)</output>', response_body.get("content")[0]["text"], re.DOTALL)[0]

try:
    FinancialMetrics.parse_raw(json_output)
    print(f"Output is valid JSON and conforms to the schema")
except Exception as e:
    print(f"Error parsing output: {e}")
    
rprint(Markdown(f'```json\n{json_output}```'))

Did the model generate the expected output? If not, what went wrong? What could you do to improve the output? 
It is important to understand that the model is not perfect and may not always generate the expected output. Therefore, you must always perform validation checks on the output to ensure that it meets your requirements. If the output is not as expected, you can try the following steps to improve the output:
- Use a larger model
- Add an automated validation step to have the model validate itself
- Perform additional prompt engineering to guide the model to generate the desired output

### PII Redaction
When working with sensitive data, it is important to ensure that any personally identifiable information (PII) is redacted from the output. Claude can be used to redact PII from a document by providing a list of PII entities to redact. 

In [ ]:

pii_extraction_template = f"""Here is some text. We want to remove all personally identifying information from this text and replace it with ****.  It's very important that names, phone numbers, and email addresses, gets replaced with ******.
Here is the text, inside <text></text> XML tags

<text>
Phone Directory:
John Latrabe, 800-232-1995,  john909709@geemail.com
Josie Lana, 800-759-2905,   josie@josielananier.com
Keven Stevens, 800-980-7000,  drkevin22@geemail.com

Phone directory will be kept up to date by the HR manager.
</text>

Please put your sanitized version of the text with PII removed in <response></response> XML tags

"""


In [ ]:

body = json.dumps({
    "max_tokens": 1024,
    "messages": [{"role": "user", "content": pii_extraction_template}],
    "anthropic_version": "bedrock-2023-05-31"
})

modelId = "anthropic.claude-3-haiku-20240307-v1:0"  # change this to use a different version from the model provider
# modelId = "anthropic.claude-3-sonnet-20240229-v1:0"  
# modelId = "anthropic.claude-v2:1" 
accept = "application/json"
contentType = "application/json"

response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("content")[0]["text"])


## One Up on Wall Street

### Challenges with reading PDF docs

Certain PDFs can not simply be read using packages like PyPDF. PDF files don't have a character encoding like text files do. They are binary files and contain many different elements, including text, images, metadata, and more. The text within a PDF can be stored in several different ways, including as plain text, as a part of the PDF's internal structure, or as a part of an embedded font. You may have to perform additional pre-processing steps such as OCR before feeding the data to the model. Below is an example where PDF extraction doesn't yield the desired text.

In [ ]:

one_up_reader = PdfReader("./data/one-up-on-wall-street-full.pdf")
one_up_number_of_pages = len(one_up_reader.pages)
one_up_text = '\n'.join([page.extract_text() for page in one_up_reader.pages])
#one_up_text = one_up_text[:50000]
print(one_up_text)


In [ ]:

# If you'd like to try your own prompt, edit this parameter!
one_up_data = f"""Here is book about Mr. Peter Lynch: 
<book>
{one_up_text}
</book>

You are a financial analyst. Please answer the question from the book only if you can find relevant context is present, if you dont have the context from the book, Say I dont know:
According to the author of the above book, what is the most important factor driving long-term value of a stock?

"""

In [ ]:
body = json.dumps({
    "max_tokens": 1024,
    "messages": [{"role": "user", "content": one_up_data}],
    "anthropic_version": "bedrock-2023-05-31"
})

modelId = "anthropic.claude-3-haiku-20240307-v1:0"  # change this to use a different version from the model provider
# modelId = "anthropic.claude-3-sonnet-20240229-v1:0"  
# modelId = "anthropic.claude-v2:1" 
accept = "application/json"
contentType = "application/json"

response = bedrock_runtime.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

print(response_body.get("content")[0]["text"])

## Conclusion

In this notebook, you got familiar with
1. Working with Claude models 
2. How to leverage Claude's long context length and usecases where this might be a best fit
3. Best practices and recommendations while using Claude models
4. Leveraging Claude in different NLP tasks using promptig